In [1]:
#IMPORTING REQUIRED LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
import json

from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import gc
gc.enable()


import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

dftrain = pd.read_csv('train.csv', dtype={'fullVisitorId':str,'date':str,'sessionId':str,'visitId':str,'visitStartTime':str})
#FUNCTION FOR READING DICTIONARY ITEMS AND HANDLING KEYERROR

#All functions

#FUNCTION FOR PROVIDING FEATURE SUMMARY
def feature_summary(df_fa):
    print('DataFrame shape')
    print('rows:',df_fa.shape[0])
    print('cols:',df_fa.shape[1])
    col_list=['Null','Unique_Count','Data_type','Max/Min','Mean','Std','Skewness','Sample_values']
    df=pd.DataFrame(index=df_fa.columns,columns=col_list)
    df['Null']=list([len(df_fa[col][df_fa[col].isnull()]) for i,col in enumerate(df_fa.columns)])
    #df['%_Null']=list([len(df_fa[col][df_fa[col].isnull()])/df_fa.shape[0]*100 for i,col in enumerate(df_fa.columns)])
    df['Unique_Count']=list([len(df_fa[col].unique()) for i,col in enumerate(df_fa.columns)])
    df['Data_type']=list([df_fa[col].dtype for i,col in enumerate(df_fa.columns)])
    for i,col in enumerate(df_fa.columns):
        if 'float' in str(df_fa[col].dtype) or 'int' in str(df_fa[col].dtype):
            df.at[col,'Max/Min']=str(round(df_fa[col].max(),2))+'/'+str(round(df_fa[col].min(),2))
            df.at[col,'Mean']=df_fa[col].mean()
            df.at[col,'Std']=df_fa[col].std()
            df.at[col,'Skewness']=df_fa[col].skew()
        df.at[col,'Sample_values']=list(df_fa[col].unique())
           
    return(df.fillna('-'))
def get_val(x,col):
    try:
        y=x[col]
    except:
        y=np.nan
    return(y)

train_len=len(dftrain)

j_fields=['device','geoNetwork','totals','trafficSource']

for col in j_fields:
    dftrain[col].replace('false','False',inplace=True,regex=True)
    dftrain[col].replace('true','True',inplace=True,regex=True)
    dftrain[col].replace('not available in demo dataset','NaN',inplace=True,regex=True)
    dftrain[col].replace('(not set)','NaN',inplace=True,regex=True)
    dftrain[col]=dftrain[col].apply(lambda x: eval(x))

device_cols=list(dftrain['device'][0].keys())
geoNetwork_cols=list(dftrain['geoNetwork'][0].keys())
totals_cols=list(dftrain['totals'][0].keys())
trafficSource_cols=list(dftrain['trafficSource'][0].keys())

for i in range(1,len(dftrain)):
    device_cols=list(set(device_cols) | set(list(dftrain['device'][i].keys())))
    geoNetwork_cols=list(set(geoNetwork_cols) | set(list(dftrain['geoNetwork'][i].keys())))
    totals_cols=list(set(totals_cols) | set(list(dftrain['totals'][i].keys())))
    trafficSource_cols=list(set(trafficSource_cols) | set(list(dftrain['trafficSource'][i].keys())))
    
for jlist in device_cols:
    col_name='device_'+jlist
    dftrain[col_name]=dftrain['device'].apply(lambda x: get_val(x,jlist))

#DROPPING device JSON FEATURE
dftrain.drop('device',axis=1,inplace=True)
#display(dftrain.head())   

#CONVERTING geoNetwork JSON FEATURE INTO INDIVIDUAL SUB FEATURES UNDER IT
for jlist in geoNetwork_cols:
    col_name='geoNetwork_'+jlist
    dftrain[col_name]=dftrain['geoNetwork'].apply(lambda x: get_val(x,jlist))
    
#DROPPING geoNetwork JSON FEATURE
dftrain.drop('geoNetwork',axis=1,inplace=True)
#display(dftrain.head())

#CONVERTING trafficSource JSON FEATURE INTO INDIVIDUAL SUB FEATURES UNDER IT
for jlist in trafficSource_cols:
    col_name='trafficSource_'+jlist
    dftrain[col_name]=dftrain['trafficSource'].apply(lambda x: get_val(x,jlist)) 


#DROPPING trafficSource JSON FEATURE
dftrain.drop('trafficSource',axis=1,inplace=True)
#display(dftrain.head())

#CONVERTING totals JSON FEATURE INTO INDIVIDUAL SUB FEATURES UNDER IT
for jlist in totals_cols:
    print(jlist)
    col_name='totals_'+jlist
    dftrain[col_name]=dftrain['totals'].apply(lambda x: get_val(x,jlist)) 
    
#DROPPING totals JSON FEATURE
dftrain.drop('totals',axis=1,inplace=True)
#display(dftrain.head())

pageviews
hits
transactionRevenue
bounces
newVisits
visits


In [2]:
#dftrain[dftrain['totals_transactionRevenue'].notnull()] #this is working
#CONVERTING ALL STRING 'NaN' VALUES TO np.nan
dftrain.replace('NaN',np.nan,inplace=True)

#CONVERTING VALUES IN FEATURE trafficSource_adwordsClickInfo AS THEY ARE AGAIN Json FEATURES
#WE HAVE TO HANDLE trafficSource_adwordsClickInfo AS OTHER Json FEATURES
dftrain['trafficSource_adwordsClickInfo']=dftrain.trafficSource_adwordsClickInfo.apply(lambda x: str(x))

def UniqueCount(df): 
    dftemp=pd.DataFrame(index=df.columns)
    dftemp['Unique_Count']=list([len(df[col].unique()) for i,col in enumerate(df.columns)])
    return(dftemp.fillna('-'))
    
#IDENFIYING ALL FEATURES WITH SAME VALUES IN ALL OBSERVATIONS
ucdf = UniqueCount(dftrain)
col_drop=list(ucdf.index[ucdf.Unique_Count==1])
print('No of Columns to be removed:',len(col_drop))
print(col_drop)

#DROPPING ALL FEATURES WITH SAME VALUES IN ALL OBSERVATIONS
dftrain.drop(col_drop,axis=1,inplace=True)
gc.collect()

dftrain.head()

No of Columns to be removed: 18
['socialEngagementType', 'device_mobileDeviceModel', 'device_operatingSystemVersion', 'device_mobileDeviceInfo', 'device_screenColors', 'device_browserSize', 'device_mobileDeviceMarketingName', 'device_screenResolution', 'device_flashVersion', 'device_browserVersion', 'device_language', 'device_mobileDeviceBranding', 'device_mobileInputSelector', 'geoNetwork_latitude', 'geoNetwork_networkLocation', 'geoNetwork_longitude', 'geoNetwork_cityId', 'totals_visits']


,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device_operatingSystem,device_isMobile,device_deviceCategory,...,trafficSource_keyword,trafficSource_medium,trafficSource_adContent,trafficSource_campaign,trafficSource_isTrueDirect,totals_pageviews,totals_hits,totals_transactionRevenue,totals_bounces,totals_newVisits
0,Organic Search,20160902,1.13166E+18,1131660440785968503_1472830385,1472830385,1,1472830385,Windows,False,desktop,...,(not provided),organic,NaN,(NaN),NaN,1,1,NaN,1,1
1,Organic Search,20160902,3.77306E+17,377306020877927890_1472880147,1472880147,1,1472880147,Macintosh,False,desktop,...,(not provided),organic,NaN,(NaN),NaN,1,1,NaN,1,1
2,Organic Search,20160902,3.89555E+18,3895546263509774583_1472865386,1472865386,1,1472865386,Windows,False,desktop,...,(not provided),organic,NaN,(NaN),NaN,1,1,NaN,1,1
3,Organic Search,20160902,4.76345E+18,4763447161404445595_1472881213,1472881213,1,1472881213,Linux,False,desktop,...,google + online,organic,NaN,(NaN),NaN,1,1,NaN,1,1
4,Organic Search,20160902,2.72944E+16,27294437909732085_1472822600,1472822600,2,1472822600,Android,True,mobile,...,(not provided),organic,NaN,(NaN),True,1,1,NaN,1,NaN


In [3]:
#DROPPING trafficSource_campaignCode, AS IT HAS ONLY 2 OBSERVATIONS WITH SOME VALUES
dftrain.drop('trafficSource_campaign',axis=1,inplace=True)

#GARBAGE COLLECTION
gc.collect()

54

In [4]:
%%time
#EXTRACTING FEATURES NAMES FROM Json FEATURE trafficSource_adwordsClickInfo
dftrain['trafficSource_adwordsClickInfo']=dftrain.trafficSource_adwordsClickInfo.apply(lambda x: eval(x))
tsadclick_cols=list(dftrain['trafficSource_adwordsClickInfo'][0].keys())

for i in range(1,len(dftrain)):
    tsadclick_cols=list(set(tsadclick_cols) | set(list(dftrain['trafficSource_adwordsClickInfo'][i].keys())))

print('FEATURES UNDER trafficSource_adwordsClickInfo Json FEATURE\n',tsadclick_cols)

#EXTRACTING FEATURES FROM Json FEATURE trafficSource_adwordsClickInfo
for jlist in tsadclick_cols:
    col_name='tsadclick_'+jlist
    dftrain[col_name]=dftrain['trafficSource_adwordsClickInfo'].apply(lambda x: get_val(x,jlist))
    dftrain[col_name]=dftrain[col_name].apply(lambda x: str(x))
    
#DROPPING Json FEATURE trafficSource_adwordsClickInfo
dftrain.drop('trafficSource_adwordsClickInfo',axis=1,inplace=True)

FEATURES UNDER trafficSource_adwordsClickInfo Json FEATURE
 ['targetingCriteria', 'adNetworkType', 'slot', 'criteriaParameters', 'isVideoAd', 'gclId', 'page']
Wall time: 47.9 s


In [5]:
#DROPPING IRRELEVANT FEATURES tsadclick_targetingCriteria AND tsadclick_criteriaParameters
dftrain.drop(['tsadclick_targetingCriteria','tsadclick_criteriaParameters'],axis=1,inplace=True)

#GARBAGE COLLECTION
gc.collect()

63

In [6]:
%%time
#REPLACING 'nan' and '(not provided)' WITH np.nan
dftrain.replace("nan",np.nan,inplace=True)
dftrain.replace("(not provided)",np.nan,inplace=True)
dftrain.replace("(NaN)",np.nan,inplace=True)
dftrain['totals_transactionRevenue'].replace(np.nan,0,inplace=True)

#CONVERTING FEATURES TO CORRECT DATATYPE
dftrain['totals_pageviews']=dftrain['totals_pageviews'].astype('float')
dftrain['totals_transactionRevenue']=dftrain['totals_transactionRevenue'].astype('float')
dftrain['totals_hits']=dftrain['totals_hits'].astype('float')
dftrain['tsadclick_page']=dftrain['tsadclick_page'].astype('float')

Wall time: 11.1 s


In [7]:
for col in dftrain.columns:
    if ((dftrain[col].dtype==object) & (col!='fullVisitorId')):
        dftrain[col]=dftrain[col].apply(lambda x: np.nan if x==np.nan else str(x).encode('utf-8', 'replace').decode('utf-8'))

In [8]:
dftrain.to_csv('normalizedtraindata.csv')